In [ ]:
using CellBasedModels
using GLMakie #Can be changes to CairoMakie
using Distributions
# using GeometryBasics
using DifferentialEquations
using Alert
using MKL
using DiffEqGPU,CUDA #use if running on NVIDIA GPU
Makie.inline!(true)

In [ ]:
CUDA.versioninfo() #Check if GPU and CUDA status

In [ ]:
function hill_f(N,K,h)
    #hill function
    if h > 0
        return N^h /(N^h + K^h)
    else
        return 1.0*(N > K)
    end
end

function gill_f(N,K,h)
    #hill function for inhibition
    if h > 0
        return K^h/(N^h + K^h)
    else
        return 1.0*(N < K)
    end
end

function gill_0(N,K)
    return 1.0*(N < K)
end

In [ ]:
#build the discrete model
model_2D3D = ABM(3,
    agent = Dict(
        :R => Float32, #Inhibitor, live only inside the agent(cell)
    ),

    model = Dict(
        :DL => Float32, #Diffusion coefficient of activator
        :aL => Float32, #Activator production rate
        :bR => Float32, #Inhibitor production rate
        :rL => Float32, #Activator degradation rate
        :rR => Float32, #Inhibitor degradation rate
        :L1 => Float32, 
        :L0 => Float32,
        :I => Float32,
        :S_on => Float32, #Initial source amplitude
        :S_duration => Float32, #Source duration
        :S_radius => Float32, #Source radius
        :S_h => Float32, #Source height
        :hill_coef_L_L => Int32,
        :hill_coef_L_R => Int32,
        :hill_coef_R_L => Int32,
    ),
    
    medium = Dict(
        :L => Float32, #Activator, live in the medium(extra-cellular)
    ),

    agentODE = quote
        dt(R) = bR*hill_f(L,L1,hill_coef_R_L) - rR*R #Inhibitor production and degradation inside the cell
    end,
    
    agentRule = quote
        # if (t < S_duration) && (x^2 + y^2)<S_radius^2
        #     L += S_on*(dt/(dx*dy*dz))
        # end
        L += (aL*hill_f(L,L0,hill_coef_L_L)*gill_f(R,I,hill_coef_L_R))*(dt/(dx*dy*dz)) #production of activator to the medium
    end,

    # agentSDE = quote
    #     dt(u) = D
    # end
    mediumODE = quote
        if @mediumInside()
            dt(L) = DL*(@∂2(1,L)+@∂2(2,L)+@∂2(3,L)) - rL*L #Diffusion and degradation of activator in the medium
        elseif @mediumBorder(1,-1)
            L = L[2,i2_,i3_] #Neumann boundary condition
        elseif @mediumBorder(1,1)
            L = L[NMedium[1]-1,i2_,i3_] 
        elseif @mediumBorder(2,-1)
            L = L[i1_,2,i3_]
        elseif @mediumBorder(2,1)
            L = L[i1_,NMedium[2]-1,i3_]
        elseif @mediumBorder(3,-1)
            L = L[i1_,i2_,2] #reflective boundary condition on the z=0 plane
        elseif @mediumBorder(3,1)
            # L = L[i1_,i2_,NMedium[3]-1] 
            L = 0 #absorbing boundary condition on the maximum z plane
        end
    end,
    mediumRule = quote
        if @mediumInside()
            if (xₘ^2 + yₘ^2)<S_radius^2 && (t < S_duration) && (abs(zₘ) < S_h)
                L += S_on*(dt/(dx*dy*dz)) #Source of activator
            end
        end
    end,
    platform=GPU(), #Switch to CPU if running on CPU
    agentAlg = DifferentialEquations.Tsit5(),
    agentSolveArgs=Dict{Symbol, Any}(:dtmin => 1e-7, :save_start => true, :abstol => 1e-6, :force_dtmin => true),
    mediumAlg = DifferentialEquations.Tsit5(),
    mediumSolveArgs = Dict{Symbol, Any}(:dtmin => 1e-7, :save_start => true,:abstol => 1e-5,:force_dtmin => false),
); 

In [ ]:
using Random
Random.seed!(1)
min_x = -400.f0 #use f0 to specify the float32 type for GPU compatibility
max_x = 400.f0
N_x = 800
paddel_x = 5.f0
min_y = -400.f0
max_y = 400.f0
N_y = 800
paddel_y = 5.f0
min_z = -1.f0
max_z = 100.f0
N_z = 101
Ncells = 7500 #number of cells in the simulation
xs = rand(Uniform(min_x,max_x),Ncells); #randomly distribute the cells in the 2D domain
ys = rand(Uniform(min_y,max_y),Ncells);
com = Community(
    model_2D3D,
    N=Ncells,
    dt=0.1f0,
    simBox=[min_x max_x;min_y max_y;min_z max_z],
    NMedium=[N_x,N_y,N_z],
)

# m = 1/100
# g = 1/10000
# d = 1
com.DL = 100.0f0
com.aL = 1200.f0
com.bR = 1.0f0
com.rR = 0.01f0
# com.rL = 5.
com.rL = 0.1f0
com.L1 = 0.01f0
com.L0 = 1.1f0
com.I = 22.f0

# com.I = 100.
com.hill_coef_L_L = 0
com.hill_coef_L_R = 0
com.hill_coef_R_L = 0
com.S_on = 20.0f0
com.S_duration = 10.f0
com.S_radius = 20.f0
com.S_h = 1.1f0

com.x = xs
com.y = ys

In [ ]:
@time evolve!(com,steps=800,saveEach=10,saveCurrentState=true) #Run the simulation
        # progressMessage=(com)->println("Step t: $(round(com.t,digits=2))"))#,progressMessage=(com)->println("Step t: $(round(com.t,digits=2))"))

In [ ]:
d = getParameter(com,[:x,:y,:R,:L,:t])
d[:L][end][:,:,N_z÷2]
# d[:L][end][:,:,N_z]

In [ ]:
#plot the simulation results
fig = Figure(resolution=(2200,400))
 
for (j,i) in enumerate(1:floor(Int,length(com)/4):length(com))
# for (j,i) in enumerate(1:floor(Int,length(com)/8):round(Int,length(com)/2))
    begin
        d = getParameter(com,[:x,:y,:R,:L,:t])

        ax = Axis(fig[1,2*j-1],title="t=$(round(d[:t][i],digits=2))")

        m = heatmap!(ax,
                range(min_x,max_x,length=size(com.L)[1]),
                range(min_y,max_y,length=size(com.L)[2]),
                d[:L][i][:,:,2],
                colorrange=(0,com.L1[1]),colormap=:thermal
            ) #plot the activator concentration in the medium as a heatmap

        xlims!(min_x,max_x)
        ylims!(min_y,max_y)

        x = Observable(d[:x][i])
        y = Observable(d[:y][i])
        # xs1 = Observable(d[:x][i]+d[:l][i]./2 .*cos.(d[:theta][i]))
        # ys1 = Observable(d[:y][i]+d[:l][i]./2 .*sin.(d[:theta][i]))
        # xs2 = Observable(d[:x][i]-d[:l][i]./2 .*cos.(d[:theta][i]))
        # ys2 = Observable(d[:y][i]-d[:l][i]./2 .*sin.(d[:theta][i]))
        # ms = Observable([Point3f0(i/2,i/2,0) for i in d[:d][i]])
        # mc = Observable([Point3f0(ll,dd/2,dd/2) for (ll,dd) in zip(d[:l][i],d[:d][i])])
        # theta = Observable(d[:theta][i])

        n = scatter!(ax,x,y,color=d[:R][i],colormap = :ice, colorrange=(0,com.I[1]),markersize=3) #plot the cells with R concentration as the color

        Colorbar(fig[1,2*j],m)
        # Colorbar(fig[1,2*j],n)
    end
end

display(fig)